In [1]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)
from sklearn.metrics import confusion_matrix
import numpy as np

# AASIST-3

In [10]:
aasist1_test = pd.read_csv("../results/AASIST1/AASIST1_AFAD_scores.txt", sep="\t")
aasist1_test['label'] = aasist1_test['label'].replace({'fake': 0, 'real': 1})
aasist1_test

/var/folders/lx/klbf168d1715rmbbfm7mswhh0000gp/T/ipykernel_23101/2876948022.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aasist1_test['label'] = aasist1_test['label'].replace({'fake': 0, 'real': 1})


,speaker_id,sentence,filename,label,gender,tts,prob_real,pred,prob_fake
0,27,يبدو الأمر شاقاً بعض الشيء من الأسفل وهو صعود ...,AFAD_F_07832,0,female,speech-2.5-hd-preview,-0.141285,1,0.609143
1,7,قال رب اجعل لي آية قال آيتك ألا تكلم الناس ثلا...,AFAD_R_01601,1,female,none,6.020185,1,0.999997
2,20,أين هو أب سامي؟,AFAD_R_04598,1,female,none,6.285375,1,0.999998
3,25,وَفِي مُحِيطِ مَسْجِدِ السَّيِّدَةِ زَيْنَبَ ب...,AFAD_R_06399,1,male,none,4.292511,1,0.999976
4,4,وَقِيلَ مَنْ رَاقٍ,AFAD_R_00965,1,male,none,1.174721,1,0.974946
...,...,...,...,...,...,...,...,...,...
1586,18,نسيت أن تطفئ المذياع البارحة ، أليس كذلك ؟,AFAD_F_04192,0,female,speech-2.5-hd-preview,0.451588,1,0.843111
1587,23,كانت عند سامي دائما حبوب في متناول اليد.,AFAD_F_05451,0,male,speech-2.5-hd-preview,0.206748,1,0.803226
1588,5,وخلق الله السماوات والأرض بالحق ولتجزى كل نفس ...,AFAD_R_01127,1,male,none,1.832918,1,0.993005
1589,26,.الكلب أعور,AFAD_F_06777,0,male,gpt-4o-mini-tts,3.370628,1,0.999720


In [11]:
aasist1_eleven = aasist1_test.query("tts in ['eleven_multilingual_v2', 'none'] and label in [1, 0]")
aasist1_openai = aasist1_test.query("tts in ['gpt-4o-mini-tts', 'none'] and label in [1, 0]")
aasist1_minimax = aasist1_test.query("tts in ['speech-2.5-hd-preview', 'none'] and label in [1, 0]")
aasist1_resemble = aasist1_test.query("tts in ['reseamble-AI', 'none'] and label in [1, 0]")

In [12]:
def compute_metrics(y_true, y_pred):
    """
    Compute precision, recall, accuracy, F1 micro and macro scores using sklearn.
    
    Parameters:
    -----------
    y_true : array-like
        Ground truth labels
    y_pred : array-like
        Predicted labels
    
    Returns:
    --------
    dict : Dictionary containing all metrics
    """
    # Compute accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Compute precision (micro and macro)
    precision_micro = precision_score(y_true, y_pred, average='micro', zero_division=0)
    precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
    
    # Compute recall (micro and macro)
    recall_micro = recall_score(y_true, y_pred, average='micro', zero_division=0)
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    
    # Compute F1 (micro and macro)
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    # Compute per-class metrics
    precision_per_class = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_per_class = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    
    # Get unique classes
    classes = np.unique(np.concatenate([y_true, y_pred]))
    
    return {
        'accuracy': accuracy,
        'precision_micro': precision_micro,
        'precision_macro': precision_macro,
        'recall_micro': recall_micro,
        'recall_macro': recall_macro,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'per_class': {
            'precision': dict(zip(classes, precision_per_class)),
            'recall': dict(zip(classes, recall_per_class)),
            'f1': dict(zip(classes, f1_per_class))
        }
    }


def save_metrics_to_file(metrics, filename='metrics_results.txt'):
    """
    Save computed metrics to a text file.
    
    Parameters:
    -----------
    metrics : dict
        Dictionary containing metrics from compute_metrics function
    filename : str
        Name of the output file (default: 'metrics_results.txt')
    """
    with open(filename, 'w') as f:
        f.write("=" * 50 + "\n")
        f.write("CLASSIFICATION METRICS RESULTS\n")
        f.write("=" * 50 + "\n\n")
        
        # Write overall metrics
        f.write("Overall Metrics:\n")
        f.write("-" * 50 + "\n")
        f.write(f"Accuracy: {metrics['accuracy']:.4f}\n\n")
        
        f.write(f"Precision (Micro): {metrics['precision_micro']:.4f}\n")
        f.write(f"Precision (Macro): {metrics['precision_macro']:.4f}\n\n")
        
        f.write(f"Recall (Micro): {metrics['recall_micro']:.4f}\n")
        f.write(f"Recall (Macro): {metrics['recall_macro']:.4f}\n\n")
        
        f.write(f"F1 Score (Micro): {metrics['f1_micro']:.4f}\n")
        f.write(f"F1 Score (Macro): {metrics['f1_macro']:.4f}\n\n")
        
        # Write per-class metrics
        f.write("=" * 50 + "\n")
        f.write("Per-Class Metrics:\n")
        f.write("=" * 50 + "\n\n")
        
        for class_label in sorted(metrics['per_class']['precision'].keys()):
            f.write(f"Class {class_label}:\n")
            f.write(f"  Precision: {metrics['per_class']['precision'][class_label]:.4f}\n")
            f.write(f"  Recall:    {metrics['per_class']['recall'][class_label]:.4f}\n")
            f.write(f"  F1 Score:  {metrics['per_class']['f1'][class_label]:.4f}\n\n")
    
    print(f"Metrics saved to {filename}")

In [13]:
save_metrics_to_file(compute_metrics(aasist1_test['label'].to_list(), aasist1_test['pred'].to_list()), '../results/AASIST1/AASIST1_AFAD_test_results.txt')
save_metrics_to_file(compute_metrics(aasist1_eleven['label'].to_list(), aasist1_eleven['pred'].to_list()), '../results/AASIST1/AASIST1_eleven_results.txt')
save_metrics_to_file(compute_metrics(aasist1_openai['label'].to_list(), aasist1_openai['pred'].to_list()), '../results/AASIST1/AASIST1_openai_results.txt')
save_metrics_to_file(compute_metrics(aasist1_minimax['label'].to_list(), aasist1_minimax['pred'].to_list()), '../results/AASIST1/AASIST1_minimax_results.txt')
save_metrics_to_file(compute_metrics(aasist1_resemble['label'].to_list(), aasist1_resemble['pred'].to_list()), '../results/AASIST1/AASIST1_resemble_results.txt')

Metrics saved to ../results/AASIST1/AASIST1_AFAD_test_results.txt
Metrics saved to ../results/AASIST1/AASIST1_eleven_results.txt
Metrics saved to ../results/AASIST1/AASIST1_openai_results.txt
Metrics saved to ../results/AASIST1/AASIST1_minimax_results.txt
Metrics saved to ../results/AASIST1/AASIST1_resemble_results.txt


# Ad-hoc

In [ ]:
fish = pd.read_csv("../results/AASIST1/AASIST1_fish_scores.txt", sep="\t")
fish['label'] = fish['label'].replace({'fake': 0, 'real': 1})
save_metrics_to_file(compute_metrics(fish['label'].to_list(), fish['pred'].to_list()), '../results/AASIST1/AASIST1_fish_test_results.txt')

Metrics saved to ../results/AASIST1/AASIST1_fish_test_results.txt


/var/folders/lx/klbf168d1715rmbbfm7mswhh0000gp/T/ipykernel_23101/2722104660.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fish['label'] = fish['label'].replace({'fake': 0, 'real': 1})


In [16]:
xtts = pd.read_csv("../results/AASIST1/AASIST1_XTTS_scores.txt", sep="\t")
xtts['label'] = xtts['label'].replace({'fake': 0, 'real': 1})
save_metrics_to_file(compute_metrics(xtts['label'].to_list(), xtts['pred'].to_list()), '../results/AASIST1/AASIST1_xtts_test_results.txt')

Metrics saved to ../results/AASIST1/AASIST1_xtts_test_results.txt


/var/folders/lx/klbf168d1715rmbbfm7mswhh0000gp/T/ipykernel_23101/2344998675.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xtts['label'] = xtts['label'].replace({'fake': 0, 'real': 1})


In [17]:
mms = pd.read_csv("../results/AASIST1/AASIST1_MMS_scores.txt", sep="\t")
mms['label'] = mms['label'].replace({'fake': 0, 'real': 1})
save_metrics_to_file(compute_metrics(mms['label'].to_list(), mms['pred'].to_list()), '../results/AASIST1/AASIST1_mms_test_results.txt')

Metrics saved to ../results/AASIST1/AASIST1_mms_test_results.txt


/var/folders/lx/klbf168d1715rmbbfm7mswhh0000gp/T/ipykernel_23101/1373641322.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mms['label'] = mms['label'].replace({'fake': 0, 'real': 1})


In [18]:
t5 = pd.read_csv("../results/AASIST1/AASIST1_T5_scores.txt", sep="\t")
t5['label'] = t5['label'].replace({'fake': 0, 'real': 1})
save_metrics_to_file(compute_metrics(t5['label'].to_list(), t5['pred'].to_list()), '../results/AASIST1/AASIST1_T5_test_results.txt')

Metrics saved to ../results/AASIST1/AASIST1_T5_test_results.txt


/var/folders/lx/klbf168d1715rmbbfm7mswhh0000gp/T/ipykernel_23101/4097739368.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  t5['label'] = t5['label'].replace({'fake': 0, 'real': 1})
